In [13]:
!pip3 install mysql-connector-python
import mysql.connector

import pandas as pd
import numpy as np

Defaulting to user installation because normal site-packages is not writeable


In [14]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="JabasR@2001",
  #database='joins'  
)

print(mydb)

mycursor = mydb.cursor(buffered=True)

# mycursor.execute("SHOW DATABASES")

# for x in mycursor:
#     print(x)


In [6]:
mycursor.execute("CREATE DATABASE Data_Spark")

In [15]:
mycursor.execute("USE Data_Spark")

In [18]:
# mycursor.execute("""
# CREATE TABLE newcustomers (
#     CustomerKey INT, 
#     Gender TEXT, 
#     Name TEXT, 
#     City TEXT, 
#     `State Code` VARCHAR(255), 
#     State TEXT, 
#     `Zip Code` TEXT,
#     Country TEXT, 
#     Continent VARCHAR(255), 
#     Birthday DATE
#     )
# """)


df = pd.read_csv('newcustomers.csv', encoding='unicode_escape')

# Insert data row by row
for i in range(len(df)):
    # Convert Birthday to datetime if it's not null, then format it to 'YYYY-MM-DD'
    birthday = pd.to_datetime(df["Birthday"].iloc[i], errors='coerce').strftime('%Y-%m-%d') if pd.notnull(df["Birthday"].iloc[i]) else None
    
    mycursor.execute("""
    INSERT INTO newcustomers (CustomerKey, Gender, Name, City, `State Code`, State, `Zip Code`, Country, Continent, Birthday)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, [
        int(df["CustomerKey"].iloc[i]),
        df["Gender"].iloc[i],
        df["Name"].iloc[i],
        df["City"].iloc[i],
        df["State Code"].iloc[i],
        df["State"].iloc[i], 
        str(df["Zip Code"].iloc[i]),  # Convert Zip Code to string
        df["Country"].iloc[i],
        df["Continent"].iloc[i],
        birthday  # Use the formatted birthday
    ])
    mydb.commit()

# Close the cursor
mycursor.close()


True

In [ ]:
# mycursor.execute("""
# CREATE TABLE Sales (
#     `Order Number` INT, 
#     `Line Item` TEXT, 
#     `Order Date` DATE, 
#     CustomerKey TEXT, 
#     StoreKey TEXT, 
#     ProductKey TEXT, 
#     Quantity TEXT, 
#     `Currency Code` TEXT
#     )
# """)

# Read data from the CSV file
df = pd.read_csv('newsales.csv', encoding='unicode_escape')

# Insert data row by row
for i in range(len(df)):
    # Convert Order Date to datetime if it's not null, then format it to 'YYYY-MM-DD'
    order_date = pd.to_datetime(df["Order Date"].iloc[i], errors='coerce').strftime('%Y-%m-%d') if pd.notnull(df["Order Date"].iloc[i]) else None
    
    # Explicitly convert numeric values to Python native types
    order_number = int(df["Order Number"].iloc[i]) if pd.notnull(df["Order Number"].iloc[i]) else None
    quantity = str(df["Quantity"].iloc[i]) if pd.notnull(df["Quantity"].iloc[i]) else None
    
    # Ensure CustomerKey, StoreKey, ProductKey are also converted to string if necessary
    customer_key = str(df["CustomerKey"].iloc[i]) if pd.notnull(df["CustomerKey"].iloc[i]) else None
    store_key = str(df["StoreKey"].iloc[i]) if pd.notnull(df["StoreKey"].iloc[i]) else None
    product_key = str(df["ProductKey"].iloc[i]) if pd.notnull(df["ProductKey"].iloc[i]) else None

    # Explicitly convert each field to its expected type
    insert_data = (
        order_number,              # Order Number (INT)
        str(df["Line Item"].iloc[i]),  # Line Item (TEXT)
        order_date,                # Order Date (DATE)
        customer_key,              # CustomerKey (TEXT)
        store_key,                 # StoreKey (TEXT)
        product_key,               # ProductKey (TEXT)
        quantity,                  # Quantity (TEXT)
        str(df["Currency Code"].iloc[i])  # Currency Code (TEXT)
    )
    
    # Execute the insert statement
    mycursor.execute("""
    INSERT INTO sales (`Order Number`, `Line Item`, `Order Date`, CustomerKey, StoreKey, ProductKey, Quantity, `Currency Code`)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, insert_data)
    
    # Commit the transaction after each row insertion
    mydb.commit()

# Close the cursor and the database connection after all inserts
mycursor.close()

In [5]:
# mycursor.execute("""
# CREATE TABLE Stores (
#     StoreKey INT,  
#     Country TEXT, 
#     State TEXT, 
#     `Square Meters` TEXT, 
#     `Open Date` DATE
# )
# """)

# Read data from the CSV file
df = pd.read_csv('newstores.csv', encoding='unicode_escape')  # Ensure the filename is correct

# Prepare a list to collect data for batch insertion
insert_data = []

# Insert data row by row
for i in range(len(df)):
    # Convert Open Date to datetime if it's not null, then format it to 'YYYY-MM-DD'
    open_date = pd.to_datetime(df["Open Date"].iloc[i], errors='coerce').strftime('%Y-%m-%d') if pd.notnull(df["Open Date"].iloc[i]) else None

    # Explicitly convert numeric values to Python native types
    store_key = int(df["StoreKey"].iloc[i]) if pd.notnull(df["StoreKey"].iloc[i]) else None
    square_meters = str(df["Square Meters"].iloc[i]) if pd.notnull(df["Square Meters"].iloc[i]) else None

    # Collect the values in a tuple
    insert_data.append((
        store_key,
        df["Country"].iloc[i],
        df["State"].iloc[i],
        square_meters,
        open_date  # Use the formatted open date
    ))

# Execute the insert statement for all rows in one go
mycursor.executemany(""" 
INSERT INTO Stores (StoreKey, Country, State, `Square Meters`, `Open Date`)
VALUES (%s, %s, %s, %s, %s)
""", insert_data)

# Commit the transaction once after all rows are inserted
mydb.commit()

# Close the cursor and the database connection
mycursor.close()


True

In [18]:
# mycursor.execute("""
# CREATE TABLE Products (
#     ProductKey INT, 
#     `Product Name` TEXT, 
#     Brand TEXT, 
#     Color TEXT, 
#     `Unit Cost USD` DECIMAL(10, 2), 
#     `Unit Price USD` DECIMAL(10, 2), 
#     SubcategoryKey TEXT,
#     Subcategory TEXT, 
#     CategoryKey VARCHAR(255), 
#     Category TEXT
# )
# """)

# Read data from the CSV file
df = pd.read_csv('newproducts.csv', encoding='unicode_escape')  # Ensure the filename is correct

# Prepare a list to collect data for batch insertion
insert_data = []

# Insert data row by row
for i in range(len(df)):
    product_key = int(df["ProductKey"].iloc[i]) if pd.notnull(df["ProductKey"].iloc[i]) else None

    # Clean the Unit Cost and Unit Price values
    unit_cost_str = str(df["Unit Cost USD"].iloc[i]).replace('$', '').strip() if pd.notnull(df["Unit Cost USD"].iloc[i]) else None
    unit_price_str = str(df["Unit Price USD"].iloc[i]).replace('$', '').strip() if pd.notnull(df["Unit Price USD"].iloc[i]) else None

    unit_cost = float(unit_cost_str) if unit_cost_str and unit_cost_str.replace('.', '', 1).isdigit() else None
    unit_price = float(unit_price_str) if unit_price_str and unit_price_str.replace('.', '', 1).isdigit() else None

    # Collect the values in a tuple, converting any pandas types to native Python types
    insert_data.append((
        product_key,
        str(df["Product Name"].iloc[i]) if pd.notnull(df["Product Name"].iloc[i]) else None,
        str(df["Brand"].iloc[i]) if pd.notnull(df["Brand"].iloc[i]) else None,
        str(df["Color"].iloc[i]) if pd.notnull(df["Color"].iloc[i]) else None,
        unit_cost,
        unit_price,
        str(df["SubcategoryKey"].iloc[i]) if pd.notnull(df["SubcategoryKey"].iloc[i]) else None,
        str(df["Subcategory"].iloc[i]) if pd.notnull(df["Subcategory"].iloc[i]) else None,
        str(df["CategoryKey"].iloc[i]) if pd.notnull(df["CategoryKey"].iloc[i]) else None,
        str(df["Category"].iloc[i]) if pd.notnull(df["Category"].iloc[i]) else None
    ))

# Execute the insert statement for all rows in one go
if insert_data:  # Ensure there's data to insert
    mycursor.executemany(""" 
    INSERT INTO Products (ProductKey, `Product Name`, Brand, Color, `Unit Cost USD`, `Unit Price USD`, SubcategoryKey, Subcategory, CategoryKey, Category)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, insert_data)

# Commit the transaction once after all rows are inserted
mydb.commit()

# Close the cursor and the database connection
mycursor.close()

True

In [21]:
mycursor.execute("""
CREATE TABLE Exchange_rates (
    Date DATE,  
    Currency TEXT, 
    Exchange TEXT 
)
""")

# Read data from the CSV file
df = pd.read_csv('exchange_rates.csv', encoding='unicode_escape')  # Ensure the filename is correct

# Prepare a list to collect data for batch insertion
insert_data = []

# Insert data row by row
for i in range(len(df)):
    date = pd.to_datetime(df["Date"].iloc[i]).date() if pd.notnull(df["Date"].iloc[i]) else None
    currency = str(df["Currency"].iloc[i]) if pd.notnull(df["Currency"].iloc[i]) else None
    exchange = str(df["Exchange"].iloc[i]) if pd.notnull(df["Exchange"].iloc[i]) else None

    # Collect the values in a tuple
    insert_data.append((date, currency, exchange))

# Check if the connection is established
if mydb.is_connected() and mycursor:  # Ensure the connection and cursor are valid
    try:
        mycursor.executemany(""" 
        INSERT INTO Exchange_rates (Date, Currency, Exchange)
        VALUES (%s, %s, %s)
        """, insert_data)
        
        # Commit the transaction once after all rows are inserted
        mydb.commit()
    except mysql.connector.Error as err:  # Handle any MySQL errors
        print(f"Error during insertion: {err}")
    finally:
        mycursor.close()  # Ensure the cursor is closed in the finally block
        mydb.close()      # Close the connection